In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from specim_test.specim.specfuncs import spec2d
from astropy.io import fits as pf

In [2]:
file_loc = '../galaxy_spectra/DEIMOS_data/PSJ1606_data/*'

In [3]:
file_list = []
out_file_det2 = []
out_file_det6 = []

for i,p in enumerate(glob.glob(file_loc)):
    if 'spec2d' in p and 'PSJ1606' in p:
        file_list.append(p)
        frame = p.split('_')[-3][:4]
        out_file_det2.append('PSJ1606_DEIMOS_spectra/PSJ1606_deimos_det2_'+ frame + '_clean.fits')
        out_file_det6.append('PSJ1606_DEIMOS_spectra/PSJ1606_deimos_det6_'+ frame + '_clean.fits')

In [4]:
file_list

['../galaxy_spectra/DEIMOS_data/PSJ1606_data/spec2d_d0613_0090-PSJ1606_DEIMOS_20210613T094724.288.fits',
 '../galaxy_spectra/DEIMOS_data/PSJ1606_data/spec2d_d0613_0091-PSJ1606_DEIMOS_20210613T101854.893.fits',
 '../galaxy_spectra/DEIMOS_data/PSJ1606_data/spec2d_d0614_0088-PSJ1606_DEIMOS_20210614T093252.253.fits']

### finding out spatial location of 2d lens spectra in science image

In [11]:
file_list_spec1d = []
frame_list = []
for i,p in enumerate(glob.glob(file_loc)):
    if 'spec1d' in p and 'PSJ1606' in p:
        file_list_spec1d.append(p)
        frame = p.split('_')[-3][:4]
        frame_list.append(frame)

In [6]:
file_list_spec1d

['../galaxy_spectra/DEIMOS_data/PSJ1606_data/spec1d_d0613_0091-PSJ1606_DEIMOS_20210613T101854.893.fits',
 '../galaxy_spectra/DEIMOS_data/PSJ1606_data/spec1d_d0613_0090-PSJ1606_DEIMOS_20210613T094724.288.fits',
 '../galaxy_spectra/DEIMOS_data/PSJ1606_data/spec1d_d0614_0088-PSJ1606_DEIMOS_20210614T093252.253.fits']

In [7]:
for fl, frm in zip(file_list_spec1d, frame_list):
    sp = pf.open(fl)
    #m = sp.slitinfo['objid']=='lens'
    nspec = sp[0].header['NSPEC']
    print(frm, nspec)
    for i in range(nspec):
        if 'lens' in sp[i+1].header['HIERARCH MASKDEF_OBJNAME']:
            print(sp[i+1].header['NAME'])
            #print(sp[i+1].header['RA'], sp[i+1].header['DEC'])

0091 155
SPAT1016-SLIT1027-DET02
SPAT1009-SLIT1022-DET06
0090 152
SPAT1015-SLIT1027-DET02
SPAT1010-SLIT1022-DET06
0088 155
SPAT1016-SLIT1027-DET02
SPAT1011-SLIT1022-DET06


## median image after sky subtraction

In [8]:
spec_det_2 = []
spec_det_6 = []
for fl in file_list:
    spec1 = spec2d.Spec2d(fl, hext=14, xtrim=[980, 1085])
    spec2 = spec2d.Spec2d(fl, hext=66, xtrim=[970, 1080])
    
    spec1.set_dispaxis('y')
    spec1.subtract_sky_2d(use_skymod=True)
    spec_det_2.append(spec1.data)
    
    spec2.set_dispaxis('y')
    spec2.subtract_sky_2d(use_skymod=True)
    spec_det_6.append(spec2.data)


Loading file ../galaxy_spectra/DEIMOS_data/PSJ1606_data/spec2d_d0613_0090-PSJ1606_DEIMOS_20210613T094724.288.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/DEIMOS_data/PSJ1606_data/spec2d_d0613_0090-PSJ1606_DEIMOS_20210613T094724.288.fits (HDU=14)
The input dataset was trimmed
 xrange: 980:1086.  yrange: 0:4096
Final data dimensions (x y): 106 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 106


Loading file ../galaxy_spectra/DEIMOS_data/PSJ1606_data/spec2d_d0613_0090-PSJ1606_DEIMOS_20210613T094724.288.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/DEIMOS_data/PSJ1606_data/spec2d_d0613_0090-PSJ1606_DEIMOS_20210613T094724.288.fits (HDU=66)
The input dataset was trimmed
 xrange: 970:1081.  yrange: 0:4096
Final data dimensions (x y): 111 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 111


Old value of dispaxis: x

Dispe

In [9]:
median_im_det2 = np.median(np.asarray(spec_det_2), axis=0)
median_im_det6 = np.median(np.asarray(spec_det_6), axis=0)

In [10]:
pf.PrimaryHDU(median_im_det2).writeto('PSJ1606_DEIMOS_spectra/PSJ1606_median_image_det2.fits')
pf.PrimaryHDU(median_im_det6).writeto('PSJ1606_DEIMOS_spectra/PSJ1606_median_image_det6.fits')

### subtract sky and cosmic rays

In [6]:
extension = [14, 66]
xtrim_list = [[980, 1085], [970, 1080]]
median_im = ['PSJ1606_DEIMOS_spectra/PSJ1606_median_image_det2.fits', 
             'PSJ1606_DEIMOS_spectra/PSJ1606_median_image_det6.fits']

for fl, ofl1, ofl2 in zip(file_list, out_file_det2, out_file_det6):
    
    for ext, xtr, mdim in zip(extension, xtrim_list, median_im):
        print(ext, xtr, mdim)
        spec = spec2d.Spec2d(fl, hext=ext, xtrim=xtr)
        #spec2 = spec2d.Spec2d(fl, hext=66, xtrim=[970, 1080])
        spec.set_dispaxis('y')
        spec.subtract_sky_2d(use_skymod=True)
        spec.szap_new(infile=mdim)
    
        filled_data = spec.data
        mask = spec.szap_mask
        #print(filled_data.shape)
        ivar_im = pf.open(fl)[ext+1].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax]
        sky_im = pf.open(fl)[ext+2].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax]
        wav_im = pf.open(fl)[ext+7].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax]
    
        ivar_im[mask] = 1.e-5
        #print(ivar_im.shape)
        
        prim_hdr = pf.open(fl)[0].header
        prim_hdr.append(('OBJECT', 'PSJ1606-2333'))
        prim_hdr.append(('', 'Trimmed and cleaned data by Pritom'))
        phdu = pf.PrimaryHDU(header=prim_hdr)
        
        sci_hdr = pf.Header()
        sci_hdr.append(('OBJECT', 'PSJ1606-2333'))
        sci_hdr.append(('IMTYPE', 'SCIENCE_IMAGE'))
        sci_im_hdu = pf.ImageHDU(data=filled_data, header=sci_hdr, name='SCI_IM')

        ivar_hdr = pf.Header()
        ivar_hdr.append(('IMTYPE', 'IVAR_IMAGE'))
        ivar_im_hdu = pf.ImageHDU(data=ivar_im, header=ivar_hdr, name='IVAR')

        sky_hdr = pf.Header()
        sky_hdr.append(('IMTYPE', 'SKY_MODEL'))
        sky_im_hdu = pf.ImageHDU(data=sky_im, header=sky_hdr, name='SKY_MOD')

        wav_hdr = pf.Header()
        wav_hdr.append(('IMTYPE', 'WAV_IMAGE'))
        wav_im_hdu = pf.ImageHDU(data=wav_im, header=wav_hdr, name='WAV_IM')

        mask_hdr = pf.Header()
        mask_hdr.append(('IMTYPE', 'MASK'))
        cmt = 'pixels masked as cosmic rays and filled with pixel values at same pixel'\
            'location from median image'
        mask_hdr.append(('', cmt))
        mask_hdu = pf.ImageHDU(data=mask, header=mask_hdr, name='MASK')

        hdul = pf.HDUList([phdu, sci_im_hdu, ivar_im_hdu, sky_im_hdu, wav_im_hdu, mask_hdu ])
        if ext==14:
            hdul.writeto(ofl1)
        else:
            hdul.writeto(ofl2)

14 [980, 1085] PSJ1606_DEIMOS_spectra/PSJ1606_median_image_det2.fits

Loading file ../galaxy_spectra/DEIMOS_data/PSJ1606_data/spec2d_d0613_0090-PSJ1606_DEIMOS_20210613T094724.288.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/DEIMOS_data/PSJ1606_data/spec2d_d0613_0090-PSJ1606_DEIMOS_20210613T094724.288.fits (HDU=14)
The input dataset was trimmed
 xrange: 980:1086.  yrange: 0:4096
Final data dimensions (x y): 106 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 106


Old value of dispaxis: x

Dispersion axis:              y
N_pixels along dispersion axis: 4096



pypeit generated sky model will be used for sky subtraction


From this point sky subtracted data will be used
 Spectrum Start:     0.00
 Spectrum End:     4095.00
 Dispersion (1st pixel):   1.00
 Dispersion (average):      1.00


From now cosmic ray rejected data will be used
66 [970, 1080] PSJ1606_DEIMOS_spectra/PSJ1606_median_image_det6.fits

